In [4]:
import astroquery.jplhorizons as jpl
import astropy.time as time
import astropy.units as u
import numpy as np
import pandas as pd
import urllib.request
import read_web
from datetime import datetime
import re
import plot

In [25]:
# Get vectors for spacecraft from JPL Horizons

def getVectors(object, start, stop, step):
    obj = jpl.Horizons(id=object, location='500@0', epochs={'start':start, 'stop':stop, 'step':step})
    return obj.vectors()

# Get vectors for object centered on current date
# plus/minus a specified number of days
# and for a specified time step
def getVectorsNow(object, start_diff, step):
    # get current date using astropy.time
    now = time.Time.now()
    # Get start/end dates
    if step.endswith('d'):
        start_diff = int(start_diff.split('d')[0])
        print(start_diff)
        start = now - start_diff*u.day
        stop = now + start_diff*u.day
    elif step.endswith('h'):
        start_diff = start_diff.split('h')[0]
        start = now - start_diff*u.hour
        stop = now + start_diff*u.hour
    elif step.endswith('m'):
        start_diff = start_diff.split('m')[0]
        start = now - start_diff*u.minute
        stop = now + start_diff*u.minute
    else:
        print('Error: time step must end with d, h, or m')
        return
    start = start.iso
    stop = stop.iso
    obj = jpl.Horizons(id=object, location='500@0', epochs={'start':start, 'stop':stop, 'step':step})
    return obj.vectors()

In [ ]:
def breakObjectList(spacecraft, minAU, maxAU):
    # If spacecraft is closer than minAU, add to freqList
    # If spacecraft is farther than maxAU, add to slowList
    # If spacecraft is between minAU and maxAU, add to midList
    freqList = []
    midList = []
    slowList = []

    for i in range(len(spacecraft)):
        if spacecraft['range'][i] < minAU:
            freqList.append(spacecraft[i])
        elif spacecraft['range'][i] > maxAU:
            slowList.append(spacecraft[i])
        else:
            midList.append(spacecraft[i])

    return freqList, midList, slowList

In [29]:
# Get current date using astropy.time
now = time.Time.now()
# Get only date from now, not time
now = now.iso.split(' ')[0]
now

'2023-09-23'

In [3]:
# Get single vector for a given object
def getSingleVector(object):
    # Get current date using astropy.time
    now = time.Time.now()
    # Get only date from now, not time
    now = now.iso.split(' ')[0]
    vector = jpl.Horizons(id=object, location='500@0', epochs={now})
    vector = vector.vectors()
    return vector

def getSingleRange(object):
    vector = getSingleVector(object)
    return vector['range'][0]

In [5]:
vec1 = getSingleVector(1)

In [17]:
x = vec1['x'][0]
y = vec1['y'][0]
z = vec1['z'][0]
print(x)

0.05082194316077274


In [5]:
planet_coords = plot.getPlanetData()
data = planet_coords.to_dict(orient='list')
print(data)
#planet_coords = [{'x': data['x'], 'y': data['y'], 'z': data['z'], 'name': planet_coords.index.values.tolist()}]
planet_coords = [{'x': data['x'], 'y': data['y']}]
# Add two nan values to the end of each list
for i in range(len(planet_coords)):
    planet_coords[i]['x'].append(float('nan'))
    planet_coords[i]['y'].append(float('nan'))
    planet_coords[i]['x'].append(float('nan'))
    planet_coords[i]['y'].append(float('nan'))

{'x': [0.05082194316077274, 0.6413879073833666, 0.9949792544456858, -1.420690979193568, 3.983655074880103, 8.793772431867284, 12.56731270048037, 29.81679557006555, 16.9230232795643], 'y': [0.2989879523036556, 0.3168004866750849, 0.004382957973220163, -0.7590205747666087, 2.954041008666241, -4.222226335472898, 15.06905568010637, -2.105765784414736, -30.4256180076543], 'z': [0.01939154639008736, -0.03289918040240307, 0.0002067771236456376, 0.01899467680009725, -0.1013810539516233, -0.2767080028477125, -0.1068450165633456, -0.6437940889592956, -1.639420121748169]}


In [10]:
planet_coords[0]['x'][2]

0.9949792544456858

In [20]:
planets = {"Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"}
# Set up dictionary to hold planet data
planetData = {}
# Populate dictionary with planet names
for planet in planets:
    planetData[planet] = [1,2,3]

planetData

{'Jupiter': [1, 2, 3],
 'Mercury': [1, 2, 3],
 'Neptune': [1, 2, 3],
 'Uranus': [1, 2, 3],
 'Mars': [1, 2, 3],
 'Saturn': [1, 2, 3],
 'Pluto': [1, 2, 3],
 'Earth': [1, 2, 3],
 'Venus': [1, 2, 3]}

In [21]:
import plot

In [32]:
planets = plot.getPlanetData()
# Switch axes of dataframe
planets = planets.transpose()



planets

,x,y,z
Mercury,0.050822,0.298988,0.019392
Venus,0.641388,0.316800,-0.032899
Earth,0.994979,0.004383,0.000207
Mars,-1.420691,-0.759021,0.018995
Jupiter,3.983655,2.954041,-0.101381
Saturn,8.793772,-4.222226,-0.276708
Uranus,12.567313,15.069056,-0.106845
Neptune,29.816796,-2.105766,-0.643794
Pluto,16.923023,-30.425618,-1.639420


In [42]:
planets.index.values.tolist()

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [44]:
data = planets.to_dict(orient='list')
result = [{'x': data['x'], 'y': data['y'], 'z': data['z'], 'name': planets.index.values.tolist()}]
result

[{'x': [0.05082194316077274,
   0.6413879073833666,
   0.9949792544456858,
   -1.420690979193568,
   3.983655074880103,
   8.793772431867284,
   12.56731270048037,
   29.81679557006555,
   16.9230232795643],
  'y': [0.2989879523036556,
   0.3168004866750849,
   0.004382957973220163,
   -0.7590205747666087,
   2.954041008666241,
   -4.222226335472898,
   15.06905568010637,
   -2.105765784414736,
   -30.4256180076543],
  'z': [0.01939154639008736,
   -0.03289918040240307,
   0.0002067771236456376,
   0.01899467680009725,
   -0.1013810539516233,
   -0.2767080028477125,
   -0.1068450165633456,
   -0.6437940889592956,
   -1.639420121748169],
  'name': ['Mercury',
   'Venus',
   'Earth',
   'Mars',
   'Jupiter',
   'Saturn',
   'Uranus',
   'Neptune',
   'Pluto']}]

In [ ]:
[{'x': [0.05082194316077274, 0.6413879073833666, 0.9949792544456858, -1.420690979193568, 3.983655074880103, 8.793772431867284, 12.56731270048037, 29.81679557006555, 16.9230232795643], 'y': [0.2989879523036556, 0.3168004866750849, 0.004382957973220163, -0.7590205747666087, 2.954041008666241, -4.222226335472898, 15.06905568010637, -2.105765784414736, -30.4256180076543], 'z': [0.01939154639008736, -0.03289918040240307, 0.0002067771236456376, 0.01899467680009725, -0.1013810539516233, -0.2767080028477125, -0.1068450165633456, -0.6437940889592956, -1.639420121748169], 'name': ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto']},
  {'x': [-30.85105845144432, 37.20203911386498], 'y': [-128.1565274780092, -99.51543441395667], 'z': [92.72856762289027, -82.48830807215792], 'name': ['Voyager 1', 'Voyager 2']}]

In [51]:
planet_coords = plot.getPlanetData()
mylist = planet_coords.to_dict(orient='list')
planet_coords

,Mercury,Venus,Earth,Mars,Jupiter,Saturn,Uranus,Neptune,Pluto
x,0.050822,0.641388,0.994979,-1.420691,3.983655,8.793772,12.567313,29.816796,16.923023
y,0.298988,0.316800,0.004383,-0.759021,2.954041,-4.222226,15.069056,-2.105766,-30.425618
z,0.019392,-0.032899,0.000207,0.018995,-0.101381,-0.276708,-0.106845,-0.643794,-1.639420


In [3]:
plot.getVoyagerData()

,x,y,z
Voyager 1,-30.851058,-128.156527,92.728568
Voyager 2,37.202039,-99.515434,-82.488308


In [6]:
planet_coords = plot.getPlanetData()
print(planet_coords)
data = planet_coords.to_dict(orient='list')
print(data)
#planet_coords = [{'x': data['x'], 'y': data['y'], 'z': data['z'], 'name': planet_coords.index.values.tolist()}]
planet_coords = [{'x': data['x'], 'y': data['y']}]
# Add two nan values to the end of each list
for i in range(len(planet_coords)):
    planet_coords[i]['x'].append(float('nan'))
    planet_coords[i]['y'].append(float('nan'))
    planet_coords[i]['x'].append(float('nan'))
    planet_coords[i]['y'].append(float('nan'))
    #planet_coords[i]['z'].append(float('nan'))
print(planet_coords)

voyager_coords = plot.getVoyagerData()
data = voyager_coords.to_dict(orient='list')
#voyager_coords = [{'x': data['x'], 'y': data['y'], 'z': data['z'], 'name': voyager_coords.index.values.tolist()}]
voyager_coords = [{'x': data['x'], 'y': data['y']}]
# Prepend 9 nan values to the beginning of each list
for j in range(9):
    for i in range(len(voyager_coords)):
        voyager_coords[i]['x'].insert(0, float('nan'))
        voyager_coords[i]['y'].insert(0, float('nan'))
        #voyager_coords[i]['z'].append(float('nan'))

object_coords = planet_coords + voyager_coords
object_coords


                 x          y         z
Mercury   0.050822   0.298988  0.019392
Venus     0.641388   0.316800 -0.032899
Earth     0.994979   0.004383  0.000207
Mars     -1.420691  -0.759021  0.018995
Jupiter   3.983655   2.954041 -0.101381
Saturn    8.793772  -4.222226 -0.276708
Uranus   12.567313  15.069056 -0.106845
Neptune  29.816796  -2.105766 -0.643794
Pluto    16.923023 -30.425618 -1.639420
{'x': [0.05082194316077274, 0.6413879073833666, 0.9949792544456858, -1.420690979193568, 3.983655074880103, 8.793772431867284, 12.56731270048037, 29.81679557006555, 16.9230232795643], 'y': [0.2989879523036556, 0.3168004866750849, 0.004382957973220163, -0.7590205747666087, 2.954041008666241, -4.222226335472898, 15.06905568010637, -2.105765784414736, -30.4256180076543], 'z': [0.01939154639008736, -0.03289918040240307, 0.0002067771236456376, 0.01899467680009725, -0.1013810539516233, -0.2767080028477125, -0.1068450165633456, -0.6437940889592956, -1.639420121748169]}
[{'x': [0.05082194316077274, 0.6

[{'x': [0.05082194316077274,
   0.6413879073833666,
   0.9949792544456858,
   -1.420690979193568,
   3.983655074880103,
   8.793772431867284,
   12.56731270048037,
   29.81679557006555,
   16.9230232795643,
   nan,
   nan],
  'y': [0.2989879523036556,
   0.3168004866750849,
   0.004382957973220163,
   -0.7590205747666087,
   2.954041008666241,
   -4.222226335472898,
   15.06905568010637,
   -2.105765784414736,
   -30.4256180076543,
   nan,
   nan]},
 {'x': [nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   -30.85105845144432,
   37.20203911386498],
  'y': [nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   nan,
   -128.1565274780092,
   -99.51543441395667]}]

In [24]:
# Convert from planets to dictionaries
# [{"x":[mercury x, venus x, ...], "y":[mercury y, venus y, ...], "z":[mercury z, venus z, ...], "name":[mercury, venus, ...]}, {"x":[mercury x, venus x, ...], "y":[mercury y, venus y, ...], "z":[mercury z, venus z, ...], "name":["Mercury", "Venus", ...]}]
def planetsToDict(planets):
    # Create list of dictionaries
    planetList = []
    # Create dictionary for each planet
    for i in range(len(planets)):
        planetDict = {}
        planetDict['x'] = planets['x'][i]
        planetDict['y'] = planets['y'][i]
        planetDict['z'] = planets['z'][i]
        planetDict['name'] = planets.index[i]
        planetList.append(planetDict)
    return planetList

planets = planetsToDict(planets)
planets

[{'x': 0.05082194316077274,
  'y': 0.2989879523036556,
  'z': 0.01939154639008736,
  'name': 'Mercury'},
 {'x': 0.6413879073833666,
  'y': 0.3168004866750849,
  'z': -0.03289918040240307,
  'name': 'Venus'},
 {'x': 0.9949792544456858,
  'y': 0.004382957973220163,
  'z': 0.0002067771236456376,
  'name': 'Earth'},
 {'x': -1.420690979193568,
  'y': -0.7590205747666087,
  'z': 0.01899467680009725,
  'name': 'Mars'},
 {'x': 3.983655074880103,
  'y': 2.954041008666241,
  'z': -0.1013810539516233,
  'name': 'Jupiter'},
 {'x': 8.793772431867284,
  'y': -4.222226335472898,
  'z': -0.2767080028477125,
  'name': 'Saturn'},
 {'x': 12.56731270048037,
  'y': 15.06905568010637,
  'z': -0.1068450165633456,
  'name': 'Uranus'},
 {'x': 29.81679557006555,
  'y': -2.105765784414736,
  'z': -0.6437940889592956,
  'name': 'Neptune'},
 {'x': 16.9230232795643,
  'y': -30.4256180076543,
  'z': -1.639420121748169,
  'name': 'Pluto'}]

In [21]:
spacecraft = ['Voyager 1', 'Voyager 2', 'Mars Express', 'Juno', 'Cassini', "Explorer 1", "Cluster 4"]

In [22]:
for craft in spacecraft:
    try :
        print(craft, getSingleRange(craft), "AU")
    except ValueError as e:
        print("1 Fail")
        if 'Ambiguous target name' in str(e):
            print(e)
            try:
                print(craft, getSingleRange(craft+' (spacecraft)'), "AU")
            except ValueError as e:
                if "No ephemeris" and "after" in str(e):
                    print("No current ephemeris for", craft)
                else:
                    print("No ephemeris found for ", craft)
        else:
            print("No ephemeris found for", craft)

Voyager 1 161.1562165650144 AU
Voyager 2 134.496458509848 AU
Mars Express 1.611846516214913 AU
1 Fail
Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
      -61  Juno (spacecraft)                                                    
     -610  Juno Centaur Stage (spacecraft)                                      
 
Juno 4.955727320890237 AU
1 Fail
Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
      -82  Cassini (spacecraft)                                                 
     -150  Cassini Huygens (spacecraft)                                         
 
No current ephemeris for Cassini
1 Fail
No ephemeris found for Explorer 1
1 Fail
No ephemeris found for Cluster 4
